
# Diplomatura en ciencia de datos, aprendizaje automático y sus aplicaciones - Edición 2023 - FAMAF (UNC)

## Análisis y visualización de datos

### Trabajo práctico entregable - Grupo 22 - Parte 1

**Integrantes:**
- Chevallier-Boutell, Ignacio José
- Ribetto, Federico Daniel
- Rosa, Santiago

**Seguimiento:** Meinardi, Vanesa


**Observación:** como los datasets son muy pequeños, nuestro flujo de trabajo se basó en hacer copias *ad hoc*. Somos concientes que esto es un gasto innecesario de memoria pero que no tiene un impacto en la performance en este caso. no nos bajen puntos porfis.

---

## Librerías

In [ ]:
import io
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import scipy

pd.set_option('display.max_rows', 1000) # cambiar el número de filas que se mostrarán usando display.max_rows.
pd.set_option('display.max_columns', 1000)
pd.set_option('display.width', 1000)
pd.options.mode.chained_assignment = None  # default='warn'

sns.set_context('talk')
sns.set_theme(style='white')

## Lectura del dataset

El dataset a utilizar es la encuesta Sysarmy del año 2022 versión 2, en formato csv, la cual es una una encuesta personal y voluntaria que busca relevar información sobre salarios y condiciones de trabajo de programadores, que se realiza anualmente. Se analizarán sólo los datos provenientes de Argentina. 

Se utilizará un dataset que ya ha tenido un pretratamiento: 
* Se eliminó el encabezado de la encuesta original.
* Se renombraron las columnas.

Este dataset está disponible en internet, desde donde lo usaremos.

In [ ]:
url = 'https://raw.githubusercontent.com/DiploDatos/AnalisisyVisualizacion/master/sysarmy_survey_2022_processed.csv'
df = pd.read_csv(url)
total_ans = len(df) # cantidad de respuestas en tel dataset

---
# Ejercicio 1 - Análisis descriptivo

## Selección de columnas

En este primer ejercicio, debemos dar respuesta a **¿Cuáles son los lenguajes de programación asociados a los mejores salarios?** Para ello, creemos que las columnas que contienen la información relevante son salary_monthly_BRUTO, salary_monthly_NETO y tools_programming_languages. Crearemos un nuevo DataFrame que contenga entonces sólo esta información. Luego, eliminamos las filas que están vacías.

**Observación:** tomaremos también las variables categóricas work_province, profile_gender y profile_studies_level junto a las variables numéricas profile_years_experience y profile_age ya que el proceso de filtrado que haremos nos sirve como puntapié para responder el ejercicio 2.

In [ ]:
relevant_columns = ["salary_monthly_BRUTO",
                    "salary_monthly_NETO",
                    "tools_programming_languages", 
                    "profile_years_experience", 
                    "profile_age", 
                    "work_province", 
                    "profile_gender", 
                    "profile_studies_level"]

df1 = df[relevant_columns] # Nuevo DataFrame

print(f'Cantidad inicial de filas: {total_ans}.')
df1 = df1.dropna(subset=relevant_columns[:3])  # Evito limpiar con profile_studies_level
print(f'Quedan {len(df1)} filas.')

In [ ]:
# Modificación de etiquetas a usar en este ejercicio
df1.rename(columns = {"salary_monthly_BRUTO":'bruto', 
                      "salary_monthly_NETO":'neto',
                      "tools_programming_languages":'lenguajes'}
                      , inplace = True)

## Análisis preliminar

La estadística descriptiva del sueldo bruto muestra que hay una gran dispersión de datos: el sueldo bruto promedio es de $363.445, pero hay gente que llega a cobrar hasta 28 millones de pesos (más de 77 veces la media). Esto se ve reflejado también en su gran desviación estándar. Vemos también que hay gente que supuestamente tiene un sueldo nulo. Algo similar se observa con el sueldo neto.

In [ ]:
print('Resultados del describe:')
print(df1[["bruto","neto"]].describe().round(0))

Esto mismo lo apreciamos gráficamente: vemos que la distribución es extremadamente asimétrica, teniendo una cola ampliamente extendida hacia la derecha. Además, considerando que el Salario Mínimo Vital y Móvil (SMVM) en Abril de 2022 es de $38.940, hay varias respuestas que caen por debajo de este valor.

In [ ]:
fig, axs = plt.subplots(1,2, figsize=(12,5))

sns.histplot(df1[["bruto","neto"]], ax=axs[0], color='steelblue')

axs[0].ticklabel_format(style='sci', axis='x', scilimits=(6,6))
axs[0].set_ylim((-5,600))
axs[0].set_xlim((-1*10**6,30*10**6))
axs[0].set_ylabel("")
axs[0].set_xlabel("salario ($)")

sns.histplot(df1[["bruto","neto"]], ax=axs[1], color='steelblue')
axs[1].ticklabel_format(style='sci', axis='x', scilimits=(4,4))
axs[1].set_ylim((-5,600))
axs[1].set_xlim((-1000,4*10**4))
axs[1].set_ylabel("")
axs[1].set_xlabel("salario ($)")

plt.show()

Analicemos ahora la relación entre el sueldo bruto y el neto. Sabemos que el sueldo bruto tiene que ser estrictamente mayor que el sueldo neto. Esto implica que, gráficamente, todos los puntos deberían caer por encima de la diagonal principal (marcada en rojo). Limpiamos entonces los datos, considerando esta condición. Notamos también que hay una serie de datos encolumnados en torno a un sueldo neto nulo.

In [ ]:
sns.pairplot(data=df1, y_vars=['bruto'], x_vars=['neto'], aspect=2, height=4)
plt.ticklabel_format(style='sci', axis='both', scilimits=(6,6))
plt.axline(xy1=(10000, 10000), xy2=(2000000, 2000000), color="red")
plt.xlabel("salario neto ($)")
plt.ylabel("salario bruto ($)")
plt.show()

## Filtrado/limpieza

Primero vamos a eliminar aquellos datos que no cumplan con la condición de que el sueldo bruto sea estrictamente mayor que el neto. Además, dada la pregunta que queremos responder, vamos a quedarnos con aquellas personas que cobran desde un SMVM hasta hasta 2 millones de pesos, inclusive, imponiendo estas condiciones sobre el sueldo neto.

In [ ]:
print(f'Con todos los datos, teníamos {len(df1)} filas.')

df_acot = df1
df_acot = df_acot[df_acot["bruto"]>df_acot["neto"]] # Filtramos bruto > neto

SMVM = 38940
df_acot = df_acot[df_acot["neto"]>=SMVM] # Filtramos desde el SMVM.
df_acot = df_acot[df_acot["neto"]<=2*1e6] # Filtramoshasta 2 millones.

print(f'Luego de filtrar, nos quedan {len(df_acot)} filas.')



Luego de filtrar, repetimos el pairplot y vemos que ahora tiene más sentido.

In [ ]:
sns.pairplot(data=df_acot, y_vars=['bruto'], x_vars=['neto'], aspect=2, height=4)
plt.ticklabel_format(style='sci', axis='both', scilimits=(6,6))
plt.axline(xy1=(10000, 10000), xy2=(2000000, 2000000), color="red")
plt.xlabel("salario neto ($)")
plt.ylabel("salario bruto ($)")
plt.show()

Al repetir los histogramas, vemos que aún tenemos una distribución bastante asimetríca con muchos puntos a derecha, tanto en bruto como en neto.

In [ ]:
fig, axs = plt.subplots(figsize=(6,5))

sns.histplot(df_acot[["bruto","neto"]], ax=axs)
axs.ticklabel_format(style='sci', axis='x', scilimits=(6,6))
axs.set_ylim((-5,350))
axs.set_xlim((0,2.05*10**6))
axs.set_xlabel("salario ($)")
axs.set_ylabel("")
plt.show()

## Selección de la porción central de la distribución

Consideramos que sería apropiado tomar el 85 % central de nuestra muestra para llevar a cabo el análisis. Como suele de ser de interés el sueldo "de bolsillo" (el neto), seguiremos en adelante considerando sólo el sueldo neto y dejando de lado el bruto.

In [ ]:
print(f'Teníamos {len(df_acot)} filas.')

df85 = df_acot[['neto', 'lenguajes', 'profile_years_experience', 
                'profile_age', 'work_province', 'profile_gender', 
                'profile_studies_level']] # Nuevo DataFrame

k = 15
percentile_inf = df85["neto"].quantile(k * 0.5 / 100)
percentile_sup = df85["neto"].quantile((100 - k * 0.5) / 100)

df85 = df85[df85["neto"] > percentile_inf]
df85 = df85[df85["neto"] < percentile_sup]

print(f'Tomando el 85% central ahora tenemos {len(df85)} filas.')


In [ ]:
fig, axs = plt.subplots(figsize=(6,5))

sns.histplot(df85["neto"], ax=axs, color='steelblue', legend=False)
axs.ticklabel_format(style='sci', axis='x', scilimits=(6,6))
axs.set_ylim((-5,320))
axs.set_xlim((38000, 0.55*10**6))
axs.set_xlabel("salario neto ($)")
axs.set_ylabel("")
axs.vlines(df85["neto"].median(), 0, np.amax(df85["neto"]), color="tab:orange", label = f'Mediana = ${df85["neto"].median():.0f}', ls='-.')
axs.vlines(df85["neto"].mean(), 0, np.amax(df85["neto"]), color="black", label = f'Promedio = ${df85["neto"].mean():.0f}', ls='-.')
axs.legend()

plt.show()

## Selección de lenguajes de programación a analizar



La columna que contiene información sobre los lenguajes de programación utilizados es `tools_prog_langs`, la cual no hemos considerado hasta ahora. Lo primero que hay que hacer es eliminar aquellas entradas cuya respuesta sea "Ninguna de las anteriores". Luego  separar las listas de lenguajes, para que queden cada uno por su cuenta, asociado al respectivo sueldo neto. En la columna `prog_lang` se encuentra cada lenguaje por separado (las filas con igual índice hacen referencia a la misma persona).

In [ ]:
# Convert the comma-separated string of languages to a list of string.
# Remove 'ninguno de los anteriores' option, spaces and training commas.
def split_languages(languages_str):
    if not isinstance(languages_str, str):
    
        return []
  
    # Remove 'other' option
    languages_str = languages_str.lower().replace('ninguno de los anteriores', '')
    # Split string into list of items
    # Remove spaces and commas for each item
    return [lang.strip().replace(',', '') for lang in languages_str.split()]

# Create a new column with the list of languages
df85.loc[:, 'cured_lang'] = df85.lenguajes\
    .apply(split_languages)

new_relevant_columns = ['neto', 'lenguajes', 'cured_lang']

# Duplicate each row of df for each programming language
# mentioned in the response.
# We only include in df_lang the columns we are going to analyze later, so we
# don't duplicate innecesary information.

# Uso la columna que me interesa
# Transformo a series. No entiendo por que hay 19 columnas.
# Desdoblado línea por línea para que se entienda.
df_lang = df85["cured_lang"].apply(pd.Series)
df_lang = df_lang.stack()
df_lang = df_lang.reset_index(level=-1, drop=True)
df_lang = df_lang.to_frame()
df_lang = df_lang.join(df85[new_relevant_columns])
df_lang = df_lang.rename(columns={0: 'prog_lang'})

print(df_lang[:5])
print(len(df_lang))

Si contamos cuántas veces aparece cada lenguaje y ordenamos de manera creciente, vemos que los 6 lenguajes más usados son sql, javascript, html, python, css y java, llevándose el 62% de la población. Éstos son los lenguajes que consideraremos en nuestro análisis.

In [ ]:
language_count = df_lang.prog_lang.value_counts()\
    .reset_index()\
    .rename(columns={'index': 'lenguaje', 'prog_lang': 'frequency'})

print(language_count[:10])

print(f'Cantidad total de entradas: {len(df_lang)}.')
top6 = np.sum(language_count["frequency"][0:6])
int_lang = language_count["lenguaje"][0:6].to_list()
print(f'Entradas en los 6 más populares: {top6}.')
print(f'Porcentaje de los 6 lenguajes más usados: {top6 * 100 / len(df_lang):.0f}%.')


## Reformulación de la pregunta inicial

La pregunta que nos hacemos ahora es la siguiente. Considerando que en Argentina el SMVM en Abril de 2022 es de $\$ 38.940$ y que los 6 lenguajes de programación más populares son sql, javascript, html, python, css y java, ¿cuales de estos lenguajes de programación están asociados a los mejores salarios de bolsillo, considerando un sueldo mensual neto desde los $\$ 38.940$ hasta los 2 millones de pesos?

A continuación construimos el DataFrame que nos servirá para responder esta pregunta.

In [ ]:
df_intlang = df_lang[df_lang.prog_lang.isin(int_lang)]

print(df_intlang[:6])
print(len(df_intlang))

## Visualizaciones

Para comparar las distribuciones del salario en función del lenguaje, elegimos utilizar visualizaciones. Dada la característica categórica de los datos analizados, consideramos usar histogramas y gráficos de cajas. 

### Respuesta con histogramas 

Como las distribuciones están sesgadas positivamente en todos los lenguajes, vemos que los promedios son mayores a las medianas. Sin embargo, podemos apreciar que en todos los lenguajes la diferencia entre estas cantidades es pequeña en comparación al rango de salarios en general. Esto se puede cuantificar mediante la siguiente cantidad:

$$
    \text{razones} = \frac{\text{media} - \text{mediana} }{\text{rango total}} * 100\%
$$

En todos los casos, como esperábamos, esta cantidad toma valores  pequeños (entre un 2\% y 3\%). Por esto, concluímos que tanto la media como la mediana son buenos indicadores de la tendencia central de estas distribuciones. Teniendo en cuenta esto, al comparar las medias podemos afirmar que, entre los 6 lenguajes de programación más populares, el mejor salario neto se consige con python.

In [ ]:
fig, axs = plt.subplots(2,3, figsize=(18,10))


m = 0
nbins=20
binrange = [38000, 0.55*10**6]

range_salary = -np.amin(df_intlang["neto"]) + np.amax(df_intlang["neto"])
ratio = []

for i in [0,1]:
    for j in [0,1,2]:
        data = df_intlang[df_intlang["prog_lang"]==int_lang[m]]["neto"]

        sns.histplot(data, bins=nbins, binrange=binrange, ax=axs[i, j], color='steelblue')
        
        median = data.median()
        mean = data.mean()
        
        axs[i, j].vlines(median, 0, 220, color="tab:orange", label = f'Mediana = ${median:.0f}', ls='-.')
        axs[i, j].vlines(mean, 0, 220, color="black", label = f'Promedio = ${mean:.0f}', ls='-.')

        axs[i, j].annotate(f'{int_lang[m]}', xy = (0.5e5, 200), fontsize=12, ha='left')
        axs[i, j].legend(frameon=False, loc='best')

        axs[i, j].ticklabel_format(style='sci', axis='x', scilimits=(5,5))
        axs[i, j].set_ylim((-1,220))
        axs[i, j].set_xlim((38000, 0.55*10**6))
        axs[i, j].set_xlabel("salario ($)")
        axs[i, j].set_ylabel("")
        
        ratio.append( round((mean-median)/range_salary * 100, 2) )
        
        m += 1

print("razones por lenguaje:")
print()
for i in range (len(int_lang)):
    print(int_lang[i]+":", str(ratio[i])+"%")
        
plt.show()

### Respuesta con gráfico de cajas

Una forma más compacta de visualizar los datos provistos es a través de gráficos de caja, los cuales muestran explícitamente los cuartiles asociados y dejan a los valores menos representativos en forma de puntos individuales.

Al igual que en el caso de histogramas, mostramos el valor medio de los salarios. Con este tipo de visualización ahora es más fácil de visualizar el hecho de que el lenguaje Python es el que mejor salario asociado tiene. También es posible apreciar que, a pesar de que el rango intercuartílico es muy parecido en todos los casos, algunos lenguajes tienen muchos más datos "outliers" que otros.

In [ ]:
fig, axs = plt.subplots(figsize=(8,6))
sns.boxplot(data=df_intlang,
            x='neto',
            y='prog_lang',
            ax=axs,
            color = 'steelblue',
            medianprops={"color": "tab:orange", "ls": "-."},
            showmeans=True,
            meanline=True,
            meanprops={"color": "black", "ls": "-."})
axs.ticklabel_format(style='plain', axis='x')
axs.set_xlabel("Salario neto ($)")
axs.set_ylabel("Lenguaje de programación")
axs.xaxis.grid(True)
plt.show()

---
# Ejercicio 2 - Densidades y varias variables

## Selección de filas y columnas

En este segundo ejercicio, debemos dar respuesta a **¿Que herramientas (prácticas y teóricas) són útiles para explorar la base, descubrir patrones, asociaciones?** 

Dado un DataFrame, podemos considerar que cada columna es un conjunto de datos, los cuales se puede modelar con el concepto matemático de **variable aleatoria**. Luego, cada dato de esa columna es una **realización** u **observación** de dicha variable aleatoria. Habrá datos o intervalo de datos más frecuentes y otros más atípicos. Esto último se modela con el concepto de **distribución** o **densidad** de la variable aleatoria. Estas son las herramientas matemáticas que nos brinda la probabilidad y la estadística.

En particular, elegimos las variables categóricas work_province y profile_gender junto a las variables numéricas salary_monthly_NETO, profile_years_experience y profile_age. Partimos desde el DataFrame df85 utilizado en el ejercicio anterior, donde ya tenemos una serie de filtros aplicados.

In [ ]:
relevant_columns = ["neto", 
                    "profile_years_experience", 
                    "profile_age", 
                    "work_province", 
                    "profile_gender",
                    "profile_studies_level"
                   ]

df2 = df85[relevant_columns] # Nuevo DataFrame

print(f'Cantidad inicial de filas: {len(df85)}.')
df2 = df2.dropna(subset=relevant_columns[:-1]) # Evito limpiar con profile_studies_level
print(f'Quedan {len(df2)} filas.')

## Filtros

### Género

Dada que la encuesta no tiene respuestas prefijadas, la categoría contiene una gran variedad de respuestas equivalentes, sólo que escritas de distinta manera. En una primera instancia, se unificaron estas categorías para evitar ambigüedades y se graficó un histograma con las nuevas etiquetas.

Observamos que hay dos categorías dominantes, que son varon_cis y mujer_cis. Utilizando una escala logarítmica en el histograma, vemos que el resto está dos órdenes de magnitud por abajo. En base a esto, tomamos el criterio de agrupar las categorías minoritarias bajo la etiqueta "diversidades" para darle más significancia estadística para futuros análisis.

In [ ]:
print("Distintas respuestas a la sección 'genero'")
print()
print("Originalmente:")
print(df2.profile_gender.unique())
print("Número de etiquetas: ",len(df2.profile_gender.unique()))

print("------------------------------------------------------")

df2_gen = df2.copy()

df2_gen.loc[:,'profile_g'] = df2_gen.profile_gender.replace(
    {'Varón Cis': 'varon_cis',
     'Varón cis': 'varon_cis',
     'Mujer': 'mujer_cis',
     'Mujer Cis': 'mujer_cis',
     'Mujer cis': 'mujer_cis',
     'Femenino': 'mujer_cis',
     'mujer': 'mujer_cis',
     'Mujer':'mujer_cis',
     'Queer':'queer',
     'Varón Trans':'varon_trans',
     'No binarie':'no_binarie',
     'Mujer Trans':'mujer_trans',
     'Fluido':'fluido',
     'Bigénero':'bigenero',
     'Gay':'gay'
    })

print("Unificando etiquetas equivalentes")
print(df2_gen.profile_g.unique())
print("Número de etiquetas: ",len(df2_gen.profile_g.unique()))
print("------------------------------------------------------")

print("Agrupando diversidades")

df2_gen2 = df2_gen.copy()
df2_gen2.loc[:,'profile_g'] = df2_gen2.profile_g.replace(
    {
     'queer':'diversidades',
     'varon_trans':'diversidades',
     'no_binarie':'diversidades',
     'mujer_trans':'diversidades',
     'fluido':'diversidades',
     'bigenero':'diversidades',
     'gay':'diversidades'
    })

print(df2_gen2.profile_g.unique())
print("Número de etiquetas: ",len(df2_gen2.profile_g.unique()))

fig, ax = plt.subplots(1,2,figsize=(12,5))
sns.countplot(x=df2_gen.profile_g, color='steelblue', ax = ax[0])
sns.countplot(x=df2_gen2.profile_g, color='steelblue', ax = ax[1])

ax[0].set_xlabel("géneros")
ax[1].set_xlabel("géneros")
ax[0].set_ylabel("")
ax[1].set_ylabel("")
ax[0].set_yscale("log")
ax[1].set_yscale("log")
ax[0].xaxis.set_tick_params(rotation=45)
ax[1].xaxis.set_tick_params(rotation=45)
plt.show()

### Provincias

Se analizaron las respuestas provistas a la sección 'work_province'. Observamos que el 88% de les programadorxs están concentrades en cuatro provincias principalmente: Ciudad Autónoma de Buenos Aires, Provincia de Buenos Aires, Córdoba y Santa Fe, las cuales fueron renombradas para mayor conveniencia. Los análisis posteriores estarán basados  sólo en estas cuatro provincias.



In [ ]:
df_prov = df2_gen2.copy()

prov_count = df_prov.work_province.value_counts()\
    .reset_index()\
    .rename(columns={'index': 'work_province', 'work_province': 'frequency'})

print(prov_count[:10])

print(f'Cantidad total de entradas: {len(df_prov)}.')
tops = np.sum(prov_count["frequency"][0:4])
int_prov = prov_count["work_province"][0:4].to_list()
print(f'Entradas en las 3 más populares: {tops}.')
print(f'Porcentaje de las 3 provincias mayoritarias: {tops * 100 / len(df_prov):.0f}%.')

#filtro
df_prov2 = df_prov[df_prov.work_province.isin(int_prov)]
#renombramos las etiquetas mayoritarias
df_prov2.loc[:,'work_prov'] = df_prov2.work_province.replace(
    {
        'Ciudad Autónoma de Buenos Aires':'caba',
        'Córdoba':'cordoba',
        'Provincia de Buenos Aires': 'bs_as',
        'Santa Fe':'santa_fe',
     })

### Edades y años de experiencia

La edad laboral mínima en Argentina es de 18 años y la edad máxima jubilatoria es de 65 años. Filtramos los datos con estos criterios. 

Además, observamos en los datos que algunas personas o empezaron a trabajar antes de los 18 años, mintieron o se equivocaron en cargar los años de experiencia. En la tercera entrada por ejemplo, esa persona empezó a trabajar a los 13 años. Decidimos filtrar todos estos casos de la siguiente forma:

$$
    \text{edad}-18 \geq \text{años de experiencia}
$$



In [ ]:
df_age = df_prov2.copy()

age_min = 18
age_max = 65

#filtro por edades laborales
df_age = df_age[df_age["profile_age"]>=age_min]
df_age = df_age[df_age["profile_age"]<=age_max]

#algunos datos sospechosos:
print(df_age[df_age["profile_age"]-age_min < df_age["profile_years_experience"]]\
      [["profile_age","profile_years_experience"]][0:10])

#filtro de datos sospechosos
df_age = df_age[df_age["profile_age"]-age_min >= df_age["profile_years_experience"]] # años de exp no superan edad

## a) Densidad conjunta

**Consigna:**

Que herramientas visuales y modelos puede utilizar para estudiar la distribución y comportamiento de sus datos? 

Elija tres variables numéricas y 2 variables categóricas. Visualice la base según varias de las variables elegidas. Puede describir de alguna forma el comportamiento de sus datos? Que herramientas utilizaría? Describa

Las variables que vamos a estudiar son:
- **Categóricas:** provincia y género.
- **Numéricas:** sueldo neto, edad y experiencia laboral.

Armamos un DataFrame que las contenga.

In [ ]:
# Me quedo sólo con las 5 columnas que me hacen falta y reemplazo los nombres para facilitar la escritura.

df_dc = df_age[['neto', 'profile_years_experience', 'profile_age', 'profile_g', 'work_prov']]

df_dc.rename(columns = {'salary_monthly_NETO':'neto', 
                        'profile_years_experience':'experiencia', 
                        'profile_age':'edad', 
                        'profile_g':'genero', 
                        'work_prov':'provincia'}
                        , inplace = True)

Omega = len(df_dc)
print(f'El universo que vamos a estudiar tiene un tamaño igual a {Omega}.')

# Esto es para que las gráficas queden más bonitas
hueord_gen = ['varon_cis', 'mujer_cis', 'diversidades']
hueord_prov = ['caba', 'bs_as', 'cordoba', 'santa_fe']

### Comparación entre las variables categóricas

Empezamos comparando las variables categóricas. Armamos entonces tablas de contingencia y las visualizamos con un mapa de calor. Notamos que una amplia mayoría (47% del total) son varones cis que viven en CABA. La gráfica refleja además la extrema preponderancia de varones cis en todas las provincias.

In [ ]:
vad_nNorm = pd.crosstab(df_dc.genero, df_dc.provincia) # No está normalizado
vad_Norm = pd.crosstab(df_dc.genero, df_dc.provincia, normalize=True) # Normalizado sobre el total

fig, axs = plt.subplots(1, 2, figsize=(12, 5))
sns.heatmap(vad_nNorm, annot=True, fmt=".0f", linewidth=.5, ax=axs[0], cmap='viridis')
axs[0].xaxis.tick_top()

sns.heatmap(vad_Norm, annot=True, fmt=".4f", linewidth=.5, ax=axs[1], cmap='viridis')
axs[1].xaxis.tick_top()

plt.show()

#### Histograma categórico

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(12, 5))
sns.countplot(x= df_dc.provincia, hue=df_dc.genero, ax=axs[0])
sns.countplot(x= df_dc.genero, hue=df_dc.provincia, ax=axs[1])

axs[0].set_yscale("log")
axs[1].set_yscale("log")

plt.show()

### Comparación entre las variables numéricas

Ahora vamos a analizar las variables numéricas. Al hacer la comparación entre todas las posibles combinaciones entre las variables numéricas vemos que:
- La relación más evidente es que a mayor edad, mayor experiencia se tiene. Sin embargo, ambas distribuciones tienen un sesgo positivo, siendo éste más marcado para la experiencia que para los años. La mayoría tiene unos 30 años de edad, pero unos 5 años de experiencia. Esto nos estaría indicando que en general las personas entran tarde a este mercado laboral.
- Al comparar el sueldo neto con la edad y la experiencia, la relación no es tan evidente debido al mar de puntos. De todas maneras, se nota que hay una mayor concentración en ciertas zonas: 
    - Hay una gran variedad de sueldos para personas que tienen hasta 20 años de experiencia. Incluso hay personas que con 5 años de experiencia ya están cobrando $ 500.000.
    - Hay una barrera más restrictiva en términos de edad, ya que hay gente de hasta unos 50 años que cobran menos de $ 300.000. 
    - Estas dos últimas observaciones nos estarían indicando el mayor peso que tiene la experiencia frente a la edad a la hora del sueldo.

In [ ]:
print('Comparaciones sin categorizar:')
sns.pairplot(data=df_dc, diag_kind='kde')

plt.show()

Si ahora clasificamos los resultados recién vistos según el género, se tiene que:
- La edad y la experiencia laboral no afecta entre estas categorías, ya que sus modas están prácticamente superpuestas.
- Los varones cis reciben los mayores sueldos, siendo más de 2.5 veces el sueldo máximo de las mujeres cis.
- Es más probable tener un sueldo mayor a $ 200.000 siendo varon cis.
- Todas las diversidades tienen sueldos menores a $ 100.000.

In [ ]:
print('Comparaciones según género:')
sns.pairplot(data=df_dc, hue='genero', hue_order=hueord_gen)

plt.show()

Finalmente, categorizando por provincias podemos decir que:
- Es más probable tener alrededor de 30 años en CABA, Bs. As. y CBA, pero alrededor de 35 años en Santa Fe. A pesar de esta diferencia, la mayoría tiene unos 5 años de experiencia, indicando que las personas de Santa Fe han ingresado más tarde a este mercado laboral.
- Es más probable cobrar un sueldo mayor a $ 200.000 en CABA que en los demás lugares.
- En Santa Fe y CBA se cobran sueldos similares, mientras que en Bs. As. es más probable tener un sueldo entre \$ 150.000 y \$ 200.000.

In [ ]:
print('Comparaciones según provincia:')
sns.pairplot(data=df_dc, hue='provincia', hue_order=hueord_prov)

plt.show()

## b) Asociación


En base a lo realizado en el ejercicio 1 (ver secciones 'Análisis preliminar' y 'Filtrado/limpieza'), notamos que la gran mayoría de los datos tienen una correlación positiva en el sentido que cuando una variable crece, la otra también lo hace. Aquellos datos que rompen esta tendencia fueron descartados en base a los criterios ya discutidos previamente. En resumen, decidimos trabajar solamente con el salario neto. 

Para mejorar futuras encuestas, agregaríamos dichos criterios, o bien pedir sólamente el salario neto, ya que creemos que a la gente le importa más saber cuánto dinero le queda 'en el bolsillo'.

## c) Densidad condicional 

Estudie la distribución del salario según el nivel de estudio.

Separe la población según el nivel de estudio (elija dos subpoblaciones numerosas) y grafique de manera comparativa ambos histogramas de la variable `'salary_monthly_NETO'`
¿Considera que ambas variables son independientes?
¿Qué analizaría al respecto?

Calcule medidas de centralización y dispersión para cada subpoblación



In [ ]:
from matplotlib import gridspec

df_estudio = df85.copy()
df_estudio.dropna(inplace=True)

niveles = df_estudio["profile_studies_level"].unique()
gs = gridspec.GridSpec(2, 8)

y_max = [130,25,50,30,25,5,2]


#grafico de los histogramas por nivel de educación
fig = plt.figure(figsize=(20,8))
ii=0
for nivel in niveles:
    data = df_estudio[df_estudio["profile_studies_level"]==nivel]["neto"]
    print(len(data))
    if ii<4:
        ax = plt.subplot(gs[0, 2 * ii:2 * ii + 2])
    else:
        ax = plt.subplot(gs[1, 2 * ii - 7:2 * ii + 2 - 7])
    plot = sns.histplot(df_estudio[df_estudio["profile_studies_level"]==nivel]["neto"], ax=ax, color = 'steelblue')
    
    median = data.median()
    mean = data.mean()
    
    ax.set_title(nivel)
    ax.set_xlabel("salario ($)")
    ax.set_ylabel("")
    ax.ticklabel_format(style='sci', axis='x', scilimits=(5,5))
    
    ax.vlines(median, 0, y_max[ii], color="tab:orange", label = f'Mediana = ${median:.0f}', ls='-.')
    ax.vlines(mean, 0, y_max[ii], color="black", label = f'Promedio = ${mean:.0f}', ls='-.')
    ax.annotate(f'{niveles[ii]}', xy = (0.5e5, 200), fontsize=12, ha='left')
    ax.legend(frameon=False, loc='best')
#     ax.annotate(f'{nivel[ii]}', xy = (1e5, 5), fontsize=12, ha='left')
    ax.legend(frameon=False, loc='best')
    
    ii+=1
    
fig.tight_layout()
plt.show()

In [ ]:
df_uni=df_estudio[df_estudio["profile_studies_level"]=='Universitario']
df_ter=df_estudio[df_estudio["profile_studies_level"]=='Terciario']

print('N uni= ',len(df_uni), ', N ter= ' , len(df_ter))

avg_uni = df_uni["neto"].mean()
med_uni = df_uni["neto"].median()
std_uni = df_uni["neto"].std()

avg_ter = df_ter["neto"].mean()
med_ter = df_ter["neto"].median()
std_ter = df_ter["neto"].std()

#print(std_uni,std_ter)

fig,ax = plt.subplots(figsize=(6,4))
sns.histplot(df_uni["neto"],ax=ax, color = 'lightsteelblue', bins=15)
sns.histplot(df_ter["neto"],ax=ax, color = 'yellowgreen', bins=15)
ax.set_ylabel('')
#ax.set_yscale('log')
ax.ticklabel_format(style='sci', axis='x', scilimits=(5,5))

ax.vlines(avg_uni, 0, 190, color="tab:blue", label = f'avg uni = ${avg_uni:.0f}', ls='-')
ax.vlines(med_uni, 0, 190, color="tab:blue", label = f'med uni = ${med_uni:.0f}', ls='--')
ax.vlines(avg_ter, 0, 190, color="tab:green", label = f'avg ter = ${avg_ter:.0f}', ls='-')
ax.vlines(med_ter, 0, 190, color="tab:green", label = f'med ter = ${med_ter:.0f}', ls='--')

ax.legend()
plt.show()

## d) Densidad Conjunta condicional

Elija dos variables numéricas y una categórica. 
Estudie la dispersión (scatterplot) de las dos variables discriminando en color por la variable categórica (ayuda: hue en seaborn)